In [1]:
import pandas as pd
import numpy as np

In [327]:
class FeatureExtractor:
    """Builds leak-free season-to-date features for NBA games.
    Usage: 
    feature_extractor = FeatureExtractor("nba_games_processed.csv")
    feature_extractor.build().get_train(True)"""

    def __init__(self, processed_games_csv: str) -> None:
        self.nba_games_df = pd.read_csv(processed_games_csv)
        self.nba_games_df["GAME_DATE"] = pd.to_datetime(self.nba_games_df["GAME_DATE"])
        self.stats = ['poss','ortg','drtg','eFG','tovr','orb%','ftr','pace']

        # Lazily built artifacts
        self.nba_games_joined = None
        self.games_by_home_w_stats = None
        self.games_by_away_w_stats = None
        self.all_games_w_stats = None
        self.feature_set = None

    # ---------- public API ----------

    def build(self) -> "FeatureExtractor":
        """Run the full pipeline and return self for chaining."""
        self._join_and_calculate_stats()
        self._split_into_home_away_df()
        self._concat_home_and_away()
        self._calculate_season_to_date_stats_and_join()
        self._create_feature_df()
        return self

    def get_train(self, use_delta_stats: bool, train_end: str = "2023-01-01") -> tuple[pd.DataFrame, pd.Series]:
        """Return X_train, y_train using a time cutoff. Use about half of data for training set by default. Using delta stats minuses each teams corresponding 
        s2d stats results in slightly greater accuracy than leaving as is as well as reduces our total features in half"""
        self._require_features()
        training_set = self.feature_set[self.feature_set["GAME_DATE"] < pd.Timestamp(train_end)]
        final_x_train = self._make_x(training_set, use_delta_stats=use_delta_stats)
        final_y_train = training_set["WL"]
        return final_x_train, final_y_train

    def get_test(self, use_delta_stats: bool, start: str = "2023-01-01", end: str = "2024-01-01") -> tuple[pd.DataFrame, pd.Series]:
        """Return X_test, y_test for a date window."""
        self._require_features()
        test_set = self.feature_set[(self.feature_set["GAME_DATE"] >= pd.Timestamp(start)) & (self.feature_set["GAME_DATE"] < pd.Timestamp(end))]
        final_x_test = self._make_x(test_set, use_delta_stats=use_delta_stats)
        final_y_test = test_set["WL"]
        return final_x_test, final_y_test
    
    # ---------- pipeline steps ----------

    def _join_and_calculate_stats(self) -> None:
        self.nba_games_joined = self._combine_team_games(self.nba_games_df)
        self.nba_games_joined = self._append_calculated_stats_to_joined_df(self.nba_games_joined)

    def _split_into_home_away_df(self) -> None:
        a_subset = ["GAME_ID", "TEAM_ABBREVIATION_A"] + [f'{s}_A' for s in self.stats]
        nba_joined_subset_a = self.nba_games_joined[a_subset]
        self.games_by_home_w_stats = self.nba_games_df.merge(nba_joined_subset_a, left_on=["GAME_ID", "TEAM_ABBREVIATION"], right_on=["GAME_ID", "TEAM_ABBREVIATION_A"])

        b_subset = ["GAME_ID", "TEAM_ABBREVIATION_B"] + [f'{s}_B' for s in self.stats]
        nba_joined_subset_b = self.nba_games_joined[b_subset]
        self.games_by_away_w_stats = self.nba_games_df.merge(nba_joined_subset_b, left_on=["GAME_ID", "TEAM_ABBREVIATION"], right_on=["GAME_ID", "TEAM_ABBREVIATION_B"])

        # Redundant columns
        self.games_by_home_w_stats = self.games_by_home_w_stats.drop(columns=["TEAM_ABBREVIATION_A"])
        self.games_by_away_w_stats = self.games_by_away_w_stats.drop(columns=["TEAM_ABBREVIATION_B"])

    def _concat_home_and_away(self) -> None:
        # Home and away dataframes must have same column names for concat
        a_to_rename = [f'{s}_A' for s in self.stats]
        b_to_rename = [f'{s}_B' for s in self.stats]
        
        self.games_by_home_w_stats = self.games_by_home_w_stats.rename(columns=dict(zip(a_to_rename, self.stats)))
        self.games_by_away_w_stats = self.games_by_away_w_stats.rename(columns=dict(zip(b_to_rename, self.stats)))

        # Perform concat
        self.all_games_w_stats = pd.concat([self.games_by_home_w_stats, self.games_by_away_w_stats])

        self.all_games_w_stats['TEAM_ID'] = self.all_games_w_stats['TEAM_ID'].astype('int64')

    def _calculate_season_to_date_stats_and_join(self) -> None:
        all_games_copy = self.all_games_w_stats.sort_values(['TEAM_ID','SEASON_ID','GAME_DATE']).copy()
        all_games_copy['TEAM_ID'] = all_games_copy['TEAM_ID'].astype('int64')
        # Use groupby transform to create expanding season to date averages for each of the relevant stats for each team
        for s in self.stats:
            all_games_copy[f'{s}_S2D'] = (all_games_copy.groupby(['TEAM_ID','SEASON_ID'], sort=False)[s].transform(lambda x: x.shift(1).expanding().mean()))
        # Reduce to relevant columns
        reduced_all_games_copy = all_games_copy[["GAME_ID", "TEAM_ABBREVIATION", "MATCHUP"] + [f'{s}_S2D' for s in self.stats]]

        # Merge home game s2d stats. Merging reduced all games into home by mathcup ensures we merge the home games only
        self.games_joined_w_s2d_stats = self.games_by_home_w_stats.merge(reduced_all_games_copy, how="inner", on=["GAME_ID", "TEAM_ABBREVIATION", "MATCHUP"])

        # Get away games from all games df with s2d stats
        away_games_w_s2d = reduced_all_games_copy[(reduced_all_games_copy["MATCHUP"].str.contains("@"))]

        # Finally merge in away games with s2d to final joined df, suffixes differentiate between home (A) and away (B)
        self.games_joined_w_s2d_stats = self.games_joined_w_s2d_stats.merge(away_games_w_s2d, on="GAME_ID", suffixes=("_A", "_B"))


    def _create_feature_df(self) -> None:
        # The feature df is the last step before splitting into training/validation/test sets. It contains all of our features, our target y column (WL) and retains relevant info for identifying the games (date, team id etc) that can be used for splitting the data
        self.feature_set = self.games_joined_w_s2d_stats[["SEASON_ID", "GAME_DATE", "GAME_ID", "TEAM_NAME", "TEAM_ABBREVIATION_A", "TEAM_ABBREVIATION_B", "MATCHUP_A", "MATCHUP_B", "WL"] + [f'{s}_S2D_A' for s in self.stats] + [f'{s}_S2D_B' for s in self.stats]]
        self.feature_set = self.feature_set.dropna()
        self.feature_set['WL'] = (self.feature_set['WL'] == 'W').astype(int)

    # ---------- helpers ----------

    def _require_features(self) -> None:
        if self.feature_set is None:
            raise RuntimeError("Call .build() before requesting train/test sets.")
    
    def _make_x(self, frame: pd.DataFrame, use_delta_stats: bool = True) -> pd.DataFrame:
        if use_delta_stats:
            # Rename B columns so a/b has identical column names so we can minus them
            X = frame[[f'{s}_S2D_A' for s in self.stats]].subtract(frame[[f'{s}_S2D_B' for s in self.stats]].rename(columns=dict(zip([f'{s}_S2D_B' for s in self.stats], [f'{s}_S2D_A' for s in self.stats]))), fill_value=0)
            X.columns = [f'DELTA_{s}_S2D' for s in self.stats]
            return X
        else:
            return frame[[f'{s}_S2D_A' for s in self.stats] + [f'{s}_S2D_B' for s in self.stats]]


    # ---------- stat calculators ----------

    # Function from the NBA docs. Combine games into 1 row so 1 row per game
    @staticmethod
    def _combine_team_games(df: pd.DataFrame, keep_method: str ='home') -> pd.DataFrame:
        '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.
    
            Parameters
            ----------
            df : Input DataFrame.
            keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
                - 'home' : Keep rows where TEAM_A is the home team.
                - 'away' : Keep rows where TEAM_A is the away team.
                - 'winner' : Keep rows where TEAM_A is the losing team.
                - 'loser' : Keep rows where TEAM_A is the winning team.
                - ``None`` : Keep all rows. Will result in an output DataFrame the same
                    length as the input DataFrame.
                    
            Returns
            -------
            result : DataFrame
        '''
        # Join every row to all others with the same game ID.
        joined = pd.merge(df, df, suffixes=['_A', '_B'],
                          on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
        # Filter out any row that is joined to itself.
        result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
        # Take action based on the keep_method flag.
        if keep_method is None:
            # Return all the rows.
            pass
        elif keep_method.lower() == 'home':
            # Keep rows where TEAM_A is the home team.
            result = result[result.MATCHUP_A.str.contains(' vs. ')]
        elif keep_method.lower() == 'away':
            # Keep rows where TEAM_A is the away team.
            result = result[result.MATCHUP_A.str.contains(' @ ')]
        elif keep_method.lower() == 'winner':
            result = result[result.WL_A == 'W']
        elif keep_method.lower() == 'loser':
            result = result[result.WL_A == 'L']
        else:
            raise ValueError(f'Invalid keep_method: {keep_method}')
        return result

    @staticmethod
    def _append_calculated_stats_to_joined_df(joined_df: pd.DataFrame) -> pd.DataFrame:
        df = joined_df.copy()
        df['poss_A'] = df['FGA_A'] + 0.44*df['FTA_A'] + df['TOV_A'] - df['OREB_A']
        df['poss_B'] = df['FGA_B'] + 0.44*df['FTA_B'] + df['TOV_B'] - df['OREB_B']
        
        df['ortg_A'] = 100 * df['PTS_A'] / df['poss_A']
        df['ortg_B'] = 100 * df['PTS_B'] / df['poss_B']
        
        df['drtg_A'] = 100 * df['PTS_B'] / df['poss_B']
        df['drtg_B'] = 100 * df['PTS_A'] / df['poss_A']
        
        df['eFG_A']  = (df['FGM_A'] + 0.5*df['FG3M_A']) / df['FGA_A']
        df['eFG_B']  = (df['FGM_B'] + 0.5*df['FG3M_B']) / df['FGA_B']
        
        df['tovr_A'] = df['TOV_A'] / df['poss_A']
        df['tovr_B'] = df['TOV_B'] / df['poss_B']
        
        
        df['orb%_A'] = df['OREB_A'] / (df['OREB_A'] + df['DREB_B'])
        df['orb%_B'] = df['OREB_B'] / (df['OREB_B'] + df['DREB_A'])
        
        df['ftr_A']  = df['FTA_A'] / df['FGA_A']
        df['ftr_B']  = df['FTA_B'] / df['FGA_B']
        
        df['pace_A'] = 48 * (df['poss_A'] / (df['MIN_A']/5))
        df['pace_B'] = 48 * (df['poss_B'] / (df['MIN_B']/5))
        return df

In [326]:
feature_extractor = FeatureExtractor("nba_games_processed.csv")
feature_extractor.build().get_train(True)

(      DELTA_poss_S2D  DELTA_ortg_S2D  DELTA_drtg_S2D  DELTA_eFG_S2D  \
 8           1.560000      -21.535866        9.961797      -0.016809   
 16         -7.880000        0.736588       17.142577       0.001051   
 17          4.080000       -4.251119       -3.438613      -0.028736   
 18         -9.640000       19.817773       25.589292       0.115334   
 19         -1.140000       20.266283       17.204292       0.108927   
 ...              ...             ...             ...            ...   
 3727        0.620603       -5.283767        3.299709      -0.011663   
 3728        5.671529       -2.400482        2.944206      -0.029480   
 3729        5.193175       -8.461628        3.201010      -0.077777   
 3730        0.335328       -3.161856       -0.053557      -0.022510   
 3731        3.575322        4.397630        3.838579       0.029654   
 
       DELTA_tovr_S2D  DELTA_orb%_S2D  DELTA_ftr_S2D  DELTA_pace_S2D  
 8           0.027086       -0.082333      -0.279787        2.4

In [ ]:
nba_games_df = pd.read_csv("nba_games_processed.csv")
nba_games_df["GAME_DATE"] = pd.to_datetime(nba_games_df["GAME_DATE"])

Ultimately, we want a dataframe that each row is a game and each column is a feature, most of which are rolling season to date averages from the previous games. The pronlem is that we need a joined game dataframe to calculate our initial stats (eg ortg, drtg for each game since some of them depend on opp), then we need an unjoined (2 row per game) dataframe to calculate the rolling averages (so we can easily just group by team and sort by date since theres a row for each team), then we need to join them back for our final feature dataframe.
So we have to:
- Join the games into a single row per game
- calculate stats for each game (we do it this way so we have both teams to calculate stats that depend on the opponents like drtg and orb%)
- split them so there's one dataframe for the home teams and one dataframe for the away teams
- Concat them so we have every game in one dataframe with 2 rows per game, one row for the home team and one row for the away team, now with our calculated stats. This way we can easily calculate rolling season to date average stats for each team
- then finally, join them again so we have our final feature dataframe with season to date statistics for each team for each game (one row per game with a/b stats)

In [89]:
# Function from the NBA docs
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [90]:
# Some of the stats we need to calculate depend on the opponent, join games into one row each to calculate them easier
nba_games_joined = combine_team_games(nba_games_df)

In [100]:
def append_calculated_stats_to_joined_df(joined_df):
    df['poss_A'] = df['FGA_A'] + 0.44*df['FTA_A'] + df['TOV_A'] - df['OREB_A']
    df['poss_B'] = df['FGA_B'] + 0.44*df['FTA_B'] + df['TOV_B'] - df['OREB_B']
    
    df['ortg_A'] = 100 * df['PTS_A'] / df['poss_A']
    df['ortg_B'] = 100 * df['PTS_B'] / df['poss_B']
    
    df['drtg_A'] = 100 * df['PTS_B'] / df['poss_B']
    df['drtg_B'] = 100 * df['PTS_A'] / df['poss_A']
    
    df['eFG_A']  = (df['FGM_A'] + 0.5*df['FG3M_A']) / df['FGA_A']
    df['eFG_B']  = (df['FGM_B'] + 0.5*df['FG3M_B']) / df['FGA_B']
    
    df['tovr_A'] = df['TOV_A'] / df['poss_A']
    df['tovr_B'] = df['TOV_B'] / df['poss_B']
    
    
    df['orb%_A'] = df['OREB_A'] / (df['OREB_A'] + df['DREB_B'])
    df['orb%_B'] = df['OREB_B'] / (df['OREB_B'] + df['DREB_A'])
    
    df['ftr_A']  = df['FTA_A'] / df['FGA_A']
    df['ftr_B']  = df['FTA_B'] / df['FGA_B']
    
    df['pace_A'] = 48 * (df['poss_A'] / (df['MIN_A']/5))
    df['pace_B'] = 48 * (df['poss_B'] / (df['MIN_B']/5))


Calculate our relevant stats for each game

In [99]:
append_calculated_stats_to_joined_df(nba_games_joined)

Split into home/away dataframes so we can concat them and calculate rolling season to date averages

In [152]:
stats = ['poss','ortg','drtg','eFG','tovr','orb%','ftr','pace']
a_subset = ["GAME_ID", "TEAM_ABBREVIATION_A"] + [f'{s}_A' for s in stats]
nba_joined_subset_a = nba_games_joined[a_subset]
games_by_home_w_stats = nba_games_df.merge(nba_joined_subset_a, left_on=["GAME_ID", "TEAM_ABBREVIATION"], right_on=["GAME_ID", "TEAM_ABBREVIATION_A"])

In [153]:
b_subset = ["GAME_ID", "TEAM_ABBREVIATION_B"] + [f'{s}_B' for s in stats]
nba_joined_subset_b = nba_games_joined[b_subset]
games_by_away_w_stats = nba_games_df.merge(nba_joined_subset_b, left_on=["GAME_ID", "TEAM_ABBREVIATION"], right_on=["GAME_ID", "TEAM_ABBREVIATION_B"])

In [161]:
games_by_home_w_stats.merge(games_by_away_w_stats[b_subset], on="GAME_ID").head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,TEAM_ABBREVIATION_A,poss_A,ortg_A,drtg_A,eFG_A,tovr_A,orb%_A,ftr_A,pace_A,TEAM_ABBREVIATION_B,poss_B,ortg_B,drtg_B,eFG_B,tovr_B,orb%_B,ftr_B,pace_B
0,22019,1.610613e+09,LAL,Los Angeles Lakers,21900443,2019-12-22,LAL vs. DEN,L,240,104,39,83,0.470,10,29,0.345,16,22,0.727,18,27,45,18,7,9,19,25,-24.0,LAL,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,DEN,96.20,133.056133,111.016225,0.548913,0.083160,0.386364,0.326087,95.800830
1,22019,1.610613e+09,MIL,Milwaukee Bucks,21900441,2019-12-22,MIL vs. IND,W,242,117,44,104,0.423,15,44,0.341,14,17,0.824,16,45,61,31,10,9,9,7,28.0,MIL,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529,IND,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000
2,22019,1.610613e+09,BOS,Boston Celtics,21900440,2019-12-22,BOS vs. CHA,W,239,119,46,88,0.523,14,32,0.438,13,16,0.813,10,47,57,25,2,9,11,24,26.0,BOS,96.04,123.906706,99.871134,0.602273,0.114536,0.285714,0.181818,96.441841,CHA,93.12,99.871134,123.906706,0.463415,0.032216,0.040816,0.280488,93.120000
3,22019,1.610613e+09,OKC,Oklahoma City Thunder,21900442,2019-12-22,OKC vs. LAC,W,241,118,46,97,0.474,9,27,0.333,17,20,0.850,16,33,49,24,10,4,12,20,6.0,OKC,101.80,115.913556,110.279638,0.520619,0.117878,0.333333,0.206186,101.377593,LAC,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937
4,22019,1.610613e+09,TOR,Toronto Raptors,21900439,2019-12-22,TOR vs. DAL,W,242,110,37,94,0.394,10,34,0.294,26,31,0.839,14,37,51,21,12,5,10,21,3.0,TOR,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471,DAL,104.88,102.021358,106.136627,0.472222,0.162090,0.274510,0.300000,104.880000


We now have all games by the home team with calculated stats and all games by away team with calculated stats
now we have to: 
- concatenate them so we have all games by all teams so we can compute rolling averages (group by (sort?) by team name and date or game id so we can go shift().rolling and get means for the stats)
- once we do that, we join again so we have joined dataframe of all games with rolling averages for both teams in game

In [163]:
# must rename both columns stats so we can concatenate
games_by_home_w_stats.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS',
       'TEAM_ABBREVIATION_A', 'poss_A', 'ortg_A', 'drtg_A', 'eFG_A', 'tovr_A',
       'orb%_A', 'ftr_A', 'pace_A'],
      dtype='object')

In [166]:
# Redundant column
games_by_home_w_stats = games_by_home_w_stats.drop(columns=["TEAM_ABBREVIATION_A"])

In [169]:
# Home and away dataframes must have same column names for concat
a_to_rename = [f'{s}_A' for s in stats]
games_by_home_w_stats = games_by_home_w_stats.rename(columns=dict(zip(a_to_rename, stats)))
games_by_home_w_stats.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,poss,ortg,drtg,eFG,tovr,orb%,ftr,pace
0,22019,1.610613e+09,LAL,Los Angeles Lakers,21900443,2019-12-22,LAL vs. DEN,L,240,104,39,83,0.470,10,29,0.345,16,22,0.727,18,27,45,18,7,9,19,25,-24.0,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000
1,22019,1.610613e+09,MIL,Milwaukee Bucks,21900441,2019-12-22,MIL vs. IND,W,242,117,44,104,0.423,15,44,0.341,14,17,0.824,16,45,61,31,10,9,9,7,28.0,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529
2,22019,1.610613e+09,BOS,Boston Celtics,21900440,2019-12-22,BOS vs. CHA,W,239,119,46,88,0.523,14,32,0.438,13,16,0.813,10,47,57,25,2,9,11,24,26.0,96.04,123.906706,99.871134,0.602273,0.114536,0.285714,0.181818,96.441841
3,22019,1.610613e+09,OKC,Oklahoma City Thunder,21900442,2019-12-22,OKC vs. LAC,W,241,118,46,97,0.474,9,27,0.333,17,20,0.850,16,33,49,24,10,4,12,20,6.0,101.80,115.913556,110.279638,0.520619,0.117878,0.333333,0.206186,101.377593
4,22019,1.610613e+09,TOR,Toronto Raptors,21900439,2019-12-22,TOR vs. DAL,W,242,110,37,94,0.394,10,34,0.294,26,31,0.839,14,37,51,21,12,5,10,21,3.0,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471


In [170]:
# Redundant colum
games_by_away_w_stats = games_by_away_w_stats.drop(columns=["TEAM_ABBREVIATION_B"])

In [172]:
# Again, rename so each df has same column names
games_by_away_w_stats = games_by_away_w_stats.rename(columns=dict(zip([f'{s}_B' for s in stats], stats)))

Now we have our dataframe with 2 rows per game with a row for the home teams and a row for the away teams. Now since we have every game from both sides we can easily calculate rolling season to date averages

In [175]:
all_games_w_stats = pd.concat([games_by_home_w_stats, games_by_away_w_stats])

In [176]:
all_games_w_stats[all_games_w_stats["GAME_ID"] == 21900443]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,poss,ortg,drtg,eFG,tovr,orb%,ftr,pace
0,22019,1.610613e+09,LAL,Los Angeles Lakers,21900443,2019-12-22,LAL vs. DEN,L,240,104,39,83,0.470,10,29,0.345,16,22,0.727,18,27,45,18,7,9,19,25,-24.0,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.68000
0,22019,1.610613e+09,DEN,Denver Nuggets,21900443,2019-12-22,DEN @ LAL,W,241,128,45,92,0.489,11,32,0.344,27,30,0.900,17,27,44,31,13,2,8,19,24.0,96.20,133.056133,111.016225,0.548913,0.083160,0.386364,0.326087,95.80083


In [304]:
all_games_w_stats[all_games_w_stats["TEAM_ABBREVIATION"] == "NYK"].sort_values(["GAME_DATE"]).head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,PF,PLUS_MINUS,poss,ortg,drtg,eFG,tovr,orb%,ftr,pace
15,22019,1610612752,NYK,New York Knicks,21900447,2019-12-23,NYK vs. WAS,L,241,115,...,25,-6.0,102.60,112.085770,115.019011,0.553763,0.136452,0.255814,0.161290,102.174274
25,22019,1610612752,NYK,New York Knicks,21900461,2019-12-26,NYK @ BKN,W,239,94,...,27,12.0,98.60,95.334686,83.810303,0.451087,0.131846,0.240741,0.163043,99.012552
37,22019,1610612752,NYK,New York Knicks,21900476,2019-12-28,NYK @ WAS,W,240,107,...,25,7.0,106.44,100.526118,93.773443,0.511765,0.197294,0.239130,0.305882,106.440000
65,22019,1610612752,NYK,New York Knicks,21900505,2020-01-01,NYK vs. POR,W,239,117,...,14,24.0,102.16,114.526233,94.320487,0.551020,0.068520,0.195652,0.142857,102.587448
79,22019,1610612752,NYK,New York Knicks,21900521,2020-01-03,NYK @ PHX,L,241,112,...,26,-8.0,107.20,104.477612,113.722517,0.500000,0.139925,0.250000,0.326087,106.755187


In [ ]:
all_games_w_stats['TEAM_ID'] = all_games_w_stats['TEAM_ID'].astype('int64')
all_games_w_stats.groupby(['TEAM_ID','SEASON_ID'], sort=False).head()

In [188]:
# Make a copy just to be safe
all_games_copy = all_games_w_stats.sort_values(['TEAM_ID','SEASON_ID','GAME_DATE']).copy()
all_games_copy['TEAM_ID'] = all_games_copy['TEAM_ID'].astype('int64')
# Use groupby transform to create expanding season to date averages for each of the relevant stats for each team
for s in stats:
    all_games_copy[f'{s}_S2D'] = (all_games_copy.groupby(['TEAM_ID','SEASON_ID'], sort=False)[s].transform(lambda x: x.shift(1).expanding().mean()))

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,poss,ortg,drtg,eFG,tovr,orb%,ftr,pace,poss_S2D,ortg_S2D,drtg_S2D,eFG_S2D,tovr_S2D,orb%_S2D,ftr_S2D,pace_S2D
9,22019,1610612737,ATL,Atlanta Hawks,21900444,2019-12-23,ATL @ CLE,L,239,118,45,95,0.474,15,42,0.357,13,17,0.765,5,32,37,23,11,7,16,20,-3.0,113.48,103.983081,103.171896,0.552632,0.140994,0.121951,0.178947,113.954812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,22019,1610612737,ATL,Atlanta Hawks,21900469,2019-12-27,ATL vs. MIL,L,239,86,33,91,0.363,12,41,0.293,8,14,0.571,8,38,46,20,10,8,18,18,-26.0,107.16,80.253826,103.016924,0.428571,0.167973,0.137931,0.153846,107.608368,113.48,103.983081,103.171896,0.552632,0.140994,0.121951,0.178947,113.954812
35,22019,1610612737,ATL,Atlanta Hawks,21900477,2019-12-28,ATL @ CHI,L,239,81,32,86,0.372,9,34,0.265,8,11,0.727,9,30,39,24,8,5,19,16,-35.0,100.84,80.325268,116.747182,0.424419,0.188417,0.191489,0.127907,101.261925,110.32,92.118453,103.094410,0.490602,0.154484,0.129941,0.166397,110.781590
55,22019,1610612737,ATL,Atlanta Hawks,21900491,2019-12-30,ATL @ ORL,W,240,101,39,81,0.481,9,29,0.310,14,17,0.824,11,41,52,21,8,4,20,20,8.0,97.48,103.610997,95.325953,0.537037,0.205170,0.275000,0.209877,97.480000,107.16,88.187392,107.645334,0.468541,0.165795,0.150457,0.153567,107.608368
83,22019,1610612737,ATL,Atlanta Hawks,21900517,2020-01-03,ATL @ BOS,L,239,106,39,93,0.419,16,45,0.356,12,15,0.800,6,36,42,26,6,5,14,24,-3.0,107.60,98.513011,103.063540,0.505376,0.130112,0.125000,0.161290,108.050209,104.74,92.043293,104.565489,0.485665,0.175639,0.181593,0.167644,105.076276


Now we have our unjoined (2 rows per game) dataframe with expanding season to date averages for every game and every team! Now we just have to merge them one last time so we can have the season to date stats for each team for each game in one row so we can send it off for modeling

In [305]:
all_games_copy.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,ftr,pace,poss_S2D,ortg_S2D,drtg_S2D,eFG_S2D,tovr_S2D,orb%_S2D,ftr_S2D,pace_S2D
6969,22024,1610612766,CHA,Charlotte Hornets,22401133,2025-04-06,CHA vs. CHI,L,240,117,...,0.400000,102.960000,100.477922,105.130360,114.499219,0.504606,0.147169,0.267985,0.223882,100.058195
6987,22024,1610612766,CHA,Charlotte Hornets,22401146,2025-04-08,CHA vs. MEM,L,239,100,...,0.283951,100.538912,100.509744,105.239412,114.681720,0.504924,0.146776,0.267363,0.226140,100.095397
6996,22024,1610612766,CHA,Charlotte Hornets,22401158,2025-04-09,CHA @ TOR,L,240,96,...,0.195122,103.040000,100.504810,105.171573,114.747474,0.505174,0.146815,0.265705,0.226871,100.101012
7021,22024,1610612766,CHA,Charlotte Hornets,22401174,2025-04-11,CHA @ BOS,L,241,94,...,0.107527,101.975104,100.536500,105.021525,114.868225,0.505186,0.147284,0.263804,0.226475,100.137749
7028,22024,1610612766,CHA,Charlotte Hornets,22401187,2025-04-13,CHA @ BOS,L,240,86,...,0.211765,91.920000,100.559506,104.858257,115.050556,0.504723,0.147516,0.263291,0.225006,100.160432


In [306]:
# Reduce to relevant columns
reduced_all_games_copy = all_games_copy[["GAME_ID", "TEAM_ABBREVIATION", "MATCHUP"] + [f'{s}_S2D' for s in stats]]

In [307]:
# Merge home game s2d stats
games_joined_w_s2d_stats = games_by_home_w_stats.merge(reduced_all_games_copy, how="inner", on=["GAME_ID", "TEAM_ABBREVIATION", "MATCHUP"])

In [308]:
games_joined_w_s2d_stats.sample(5)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,ftr,pace,poss_S2D,ortg_S2D,drtg_S2D,eFG_S2D,tovr_S2D,orb%_S2D,ftr_S2D,pace_S2D
2278,22021,1.610613e+09,CLE,Cleveland Cavaliers,22100404,2021-12-13,CLE vs. MIA,W,241,105,...,0.222222,92.534440,98.775714,107.311874,103.130678,0.531005,0.149126,0.231438,0.236934,98.708478
2632,22021,1.610613e+09,CHI,Chicago Bulls,22100769,2022-02-01,CHI vs. ORL,W,239,126,...,0.322222,101.181590,99.721633,111.317164,109.487997,0.541398,0.124968,0.203401,0.240722,99.696425
1692,22020,1.610613e+09,LAC,LA Clippers,22001002,2021-05-06,LAC vs. LAL,W,239,118,...,0.337662,101.864435,98.077576,116.411108,109.732625,0.566299,0.125010,0.225975,0.223885,98.039156
1016,22020,1.610613e+09,NOP,New Orleans Pelicans,22000335,2021-02-03,NOP vs. PHX,W,241,123,...,0.233333,93.848963,102.157895,108.189526,111.711994,0.527837,0.143011,0.255813,0.286804,101.612647
4631,22023,1.610613e+09,ATL,Atlanta Hawks,22300175,2023-11-11,ATL vs. MIA,L,240,109,...,0.206897,100.920000,104.905000,116.343390,112.699440,0.542917,0.136071,0.282553,0.302261,104.901360


In [224]:
# Get away games from all games df with s2d stats
away_games_w_s2d = reduced_all_games_copy[(reduced_all_games_copy["MATCHUP"].str.contains("@"))]

In [225]:
away_games_w_s2d.head()

,GAME_ID,TEAM_ABBREVIATION,MATCHUP,poss_S2D,ortg_S2D,drtg_S2D,eFG_S2D,tovr_S2D,orb%_S2D,ftr_S2D,pace_S2D
9,21900444,ATL,ATL @ CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,21900477,ATL,ATL @ CHI,110.32,92.118453,103.094410,0.490602,0.154484,0.129941,0.166397,110.781590
55,21900491,ATL,ATL @ ORL,107.16,88.187392,107.645334,0.468541,0.165795,0.150457,0.153567,107.608368
83,21900517,ATL,ATL @ BOS,104.74,92.043293,104.565489,0.485665,0.175639,0.181593,0.167644,105.076276
135,21900567,ATL,ATL @ WAS,104.76,100.155185,108.473299,0.502735,0.145141,0.170144,0.224356,104.976641


In [309]:
# Finally merge in away games with s2d to final joined df
games_joined_w_s2d_stats = games_joined_w_s2d_stats.merge(away_games_w_s2d, on="GAME_ID", suffixes=("_A", "_B"))

In [310]:
games_joined_w_s2d_stats[games_joined_w_s2d_stats["GAME_ID"] == 21900672]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION_A,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP_A,WL,MIN,PTS,...,TEAM_ABBREVIATION_B,MATCHUP_B,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B
234,22019,1.610613e+09,NYK,New York Knicks,21900672,2020-01-24,NYK vs. TOR,L,240,112,...,TOR,TOR @ NYK,101.92,109.363489,103.247451,0.532291,0.133501,0.233237,0.235019,100.548082


In [248]:
# Final df with relevant stats
feature_set = games_joined_w_s2d_stats[["SEASON_ID", "GAME_DATE", "GAME_ID", "TEAM_NAME", "TEAM_ABBREVIATION_A", "TEAM_ABBREVIATION_B", "MATCHUP_A", "MATCHUP_B", "WL"] + [f'{s}_S2D_A' for s in stats] + [f'{s}_S2D_B' for s in stats]]

In [249]:
feature_set = feature_set.dropna()

In [251]:
feature_set.head()

,SEASON_ID,GAME_DATE,GAME_ID,TEAM_NAME,TEAM_ABBREVIATION_A,TEAM_ABBREVIATION_B,MATCHUP_A,MATCHUP_B,WL,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B
8,22019,2019-12-23,21900446,Indiana Pacers,IND,TOR,IND vs. TOR,TOR @ IND,W,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471
16,22019,2019-12-25,21900458,Los Angeles Lakers,LAL,LAC,LAL vs. LAC,LAC @ LAL,L,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937
17,22019,2019-12-25,21900457,Golden State Warriors,GSW,HOU,GSW vs. HOU,HOU @ GSW,W,106.20,106.403013,97.414762,0.505747,0.178908,0.295455,0.344828,106.644351,102.12,110.654132,100.853375,0.534483,0.107716,0.136364,0.264368,101.696266
18,22019,2019-12-25,21900456,Philadelphia 76ers,PHI,MIL,PHI vs. MIL,MIL @ PHI,W,94.84,131.800928,110.190053,0.610526,0.126529,0.414634,0.115789,94.840000,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529
19,22019,2019-12-25,21900459,Denver Nuggets,DEN,NOP,DEN vs. NOP,NOP @ DEN,L,97.90,123.254974,109.433492,0.598174,0.147002,0.278896,0.259197,97.908783,99.04,102.988691,92.229199,0.489247,0.121163,0.245283,0.172043,99.454393


In [259]:
# Use about half of data for training set
training_set = feature_set[feature_set["GAME_DATE"] < "2023"]

In [260]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3644 entries, 8 to 3731
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SEASON_ID            3644 non-null   int64         
 1   GAME_DATE            3644 non-null   datetime64[ns]
 2   GAME_ID              3644 non-null   int64         
 3   TEAM_NAME            3644 non-null   object        
 4   TEAM_ABBREVIATION_A  3644 non-null   object        
 5   TEAM_ABBREVIATION_B  3644 non-null   object        
 6   MATCHUP_A            3644 non-null   object        
 7   MATCHUP_B            3644 non-null   object        
 8   WL                   3644 non-null   object        
 9   poss_S2D_A           3644 non-null   float64       
 10  ortg_S2D_A           3644 non-null   float64       
 11  drtg_S2D_A           3644 non-null   float64       
 12  eFG_S2D_A            3644 non-null   float64       
 13  tovr_S2D_A           3644 non-null   f

In [263]:
training_set.head()

,SEASON_ID,GAME_DATE,GAME_ID,TEAM_NAME,TEAM_ABBREVIATION_A,TEAM_ABBREVIATION_B,MATCHUP_A,MATCHUP_B,WL,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B
8,22019,2019-12-23,21900446,Indiana Pacers,IND,TOR,IND vs. TOR,TOR @ IND,W,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471
16,22019,2019-12-25,21900458,Los Angeles Lakers,LAL,LAC,LAL vs. LAC,LAC @ LAL,L,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937
17,22019,2019-12-25,21900457,Golden State Warriors,GSW,HOU,GSW vs. HOU,HOU @ GSW,W,106.20,106.403013,97.414762,0.505747,0.178908,0.295455,0.344828,106.644351,102.12,110.654132,100.853375,0.534483,0.107716,0.136364,0.264368,101.696266
18,22019,2019-12-25,21900456,Philadelphia 76ers,PHI,MIL,PHI vs. MIL,MIL @ PHI,W,94.84,131.800928,110.190053,0.610526,0.126529,0.414634,0.115789,94.840000,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529
19,22019,2019-12-25,21900459,Denver Nuggets,DEN,NOP,DEN vs. NOP,NOP @ DEN,L,97.90,123.254974,109.433492,0.598174,0.147002,0.278896,0.259197,97.908783,99.04,102.988691,92.229199,0.489247,0.121163,0.245283,0.172043,99.454393


In [264]:
# Get y value (whether or not home team won the game) as 1/0
training_set['win_int'] = (training_set['WL'] == 'W').astype(int)
training_set.head()

C:\Users\Jon\AppData\Local\Temp\ipykernel_40464\2597983711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_set['win_int'] = (training_set['WL'] == 'W').astype(int)


,SEASON_ID,GAME_DATE,GAME_ID,TEAM_NAME,TEAM_ABBREVIATION_A,TEAM_ABBREVIATION_B,MATCHUP_A,MATCHUP_B,WL,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B,win_int
8,22019,2019-12-23,21900446,Indiana Pacers,IND,TOR,IND vs. TOR,TOR @ IND,W,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471,1
16,22019,2019-12-25,21900458,Los Angeles Lakers,LAL,LAC,LAL vs. LAC,LAC @ LAL,L,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937,0
17,22019,2019-12-25,21900457,Golden State Warriors,GSW,HOU,GSW vs. HOU,HOU @ GSW,W,106.20,106.403013,97.414762,0.505747,0.178908,0.295455,0.344828,106.644351,102.12,110.654132,100.853375,0.534483,0.107716,0.136364,0.264368,101.696266,1
18,22019,2019-12-25,21900456,Philadelphia 76ers,PHI,MIL,PHI vs. MIL,MIL @ PHI,W,94.84,131.800928,110.190053,0.610526,0.126529,0.414634,0.115789,94.840000,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529,1
19,22019,2019-12-25,21900459,Denver Nuggets,DEN,NOP,DEN vs. NOP,NOP @ DEN,L,97.90,123.254974,109.433492,0.598174,0.147002,0.278896,0.259197,97.908783,99.04,102.988691,92.229199,0.489247,0.121163,0.245283,0.172043,99.454393,0


In [262]:
final_x_train = training_set[[f'{s}_S2D_A' for s in stats] + [f'{s}_S2D_B' for s in stats]]

In [265]:
final_y_train = training_set["win_int"]

In [266]:
final_x_train.head()

,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B
8,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471
16,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937
17,106.20,106.403013,97.414762,0.505747,0.178908,0.295455,0.344828,106.644351,102.12,110.654132,100.853375,0.534483,0.107716,0.136364,0.264368,101.696266
18,94.84,131.800928,110.190053,0.610526,0.126529,0.414634,0.115789,94.840000,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529
19,97.90,123.254974,109.433492,0.598174,0.147002,0.278896,0.259197,97.908783,99.04,102.988691,92.229199,0.489247,0.121163,0.245283,0.172043,99.454393


In [267]:
final_y_train.head()

8     1
16    0
17    1
18    1
19    0
Name: win_int, dtype: int64

In [268]:
final_x_train.to_csv("x_train.csv", index=False)
final_y_train.to_csv("y_train.csv", index=False)

In [275]:
val_set = feature_set[(feature_set["GAME_DATE"] > "2023") & (feature_set["GAME_DATE"] < "2024")]

In [277]:
val_set['win_int'] = (val_set['WL'] == 'W').astype(int)
val_set.head()

C:\Users\Jon\AppData\Local\Temp\ipykernel_40464\4056833670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_set['win_int'] = (val_set['WL'] == 'W').astype(int)


,SEASON_ID,GAME_DATE,GAME_ID,TEAM_NAME,TEAM_ABBREVIATION_A,TEAM_ABBREVIATION_B,MATCHUP_A,MATCHUP_B,WL,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B,win_int
3735,22022,2023-01-02,22200551,Charlotte Hornets,CHA,LAL,CHA vs. LAL,LAL @ CHA,L,104.367568,106.650756,113.143758,0.503541,0.129814,0.263795,0.251614,102.736492,104.706667,110.949946,112.776430,0.541162,0.135685,0.213276,0.284582,103.745366,0
3736,22022,2023-01-02,22200552,Cleveland Cavaliers,CLE,CHI,CLE vs. CHI,CHI @ CLE,W,98.761081,112.788529,107.571886,0.552899,0.138709,0.238632,0.278573,97.019862,102.125556,111.497903,111.807346,0.546542,0.132665,0.200449,0.258178,100.923870,1
3737,22022,2023-01-02,22200557,Minnesota Timberwolves,MIN,DEN,MIN vs. DEN,DEN @ MIN,W,103.674595,110.400770,111.294967,0.555235,0.149340,0.213035,0.277334,103.349111,100.805556,115.708652,113.478566,0.581218,0.144647,0.237261,0.267618,100.161120,1
3738,22022,2023-01-02,22200553,Indiana Pacers,IND,TOR,IND vs. TOR,TOR @ IND,W,103.934054,111.163394,113.241497,0.547383,0.138290,0.220091,0.260135,103.794620,100.268889,111.076204,112.239870,0.511093,0.119262,0.270276,0.281170,99.680484,1
3739,22022,2023-01-02,22200556,Houston Rockets,HOU,DAL,HOU vs. DAL,DAL @ HOU,L,102.291111,106.929396,114.261313,0.510105,0.159880,0.302569,0.291661,101.605292,98.967568,113.826442,111.994620,0.563363,0.120754,0.188180,0.309092,97.604704,0


In [278]:
x_val = val_set[[f'{s}_S2D_A' for s in stats] + [f'{s}_S2D_B' for s in stats]]
y_val = val_set["win_int"]

In [281]:
x_val.to_csv("x_val_set.csv", index=False)
y_val.to_csv("y_val_set.csv", index=False)

In [282]:
feature_set.head()

,SEASON_ID,GAME_DATE,GAME_ID,TEAM_NAME,TEAM_ABBREVIATION_A,TEAM_ABBREVIATION_B,MATCHUP_A,MATCHUP_B,WL,poss_S2D_A,ortg_S2D_A,drtg_S2D_A,eFG_S2D_A,tovr_S2D_A,orb%_S2D_A,ftr_S2D_A,pace_S2D_A,poss_S2D_B,ortg_S2D_B,drtg_S2D_B,eFG_S2D_B,tovr_S2D_B,orb%_S2D_B,ftr_S2D_B,pace_S2D_B
8,22019,2019-12-23,21900446,Indiana Pacers,IND,TOR,IND vs. TOR,TOR @ IND,W,105.20,84.600760,111.983155,0.430000,0.123574,0.181818,0.050000,105.200000,103.64,106.136627,102.021358,0.446809,0.096488,0.264151,0.329787,102.783471
16,22019,2019-12-25,21900458,Los Angeles Lakers,LAL,LAC,LAL vs. LAC,LAC @ LAL,L,93.68,111.016225,133.056133,0.530120,0.202818,0.400000,0.265060,93.680000,101.56,110.279638,115.913556,0.529070,0.147696,0.232558,0.279070,101.984937
17,22019,2019-12-25,21900457,Golden State Warriors,GSW,HOU,GSW vs. HOU,HOU @ GSW,W,106.20,106.403013,97.414762,0.505747,0.178908,0.295455,0.344828,106.644351,102.12,110.654132,100.853375,0.534483,0.107716,0.136364,0.264368,101.696266
18,22019,2019-12-25,21900456,Philadelphia 76ers,PHI,MIL,PHI vs. MIL,MIL @ PHI,W,94.84,131.800928,110.190053,0.610526,0.126529,0.414634,0.115789,94.840000,104.48,111.983155,84.600760,0.495192,0.086141,0.296296,0.163462,103.616529
19,22019,2019-12-25,21900459,Denver Nuggets,DEN,NOP,DEN vs. NOP,NOP @ DEN,L,97.90,123.254974,109.433492,0.598174,0.147002,0.278896,0.259197,97.908783,99.04,102.988691,92.229199,0.489247,0.121163,0.245283,0.172043,99.454393


In [295]:
# Create delta feature set where stats for each team are subtracted from each other
stat_delta_training_set = training_set[[f'{s}_S2D_A' for s in stats]].subtract(training_set[[f'{s}_S2D_B' for s in stats]].rename(columns=dict(zip([f'{s}_S2D_B' for s in stats], [f'{s}_S2D_A' for s in stats]))), fill_value=0)

In [299]:
stat_delta_val_set = val_set[[f'{s}_S2D_A' for s in stats]].subtract(val_set[[f'{s}_S2D_B' for s in stats]].rename(columns=dict(zip([f'{s}_S2D_B' for s in stats], [f'{s}_S2D_A' for s in stats]))), fill_value=0)

In [297]:
stat_delta_training_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3644 entries, 8 to 3731
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   poss_S2D_A  3644 non-null   float64
 1   ortg_S2D_A  3644 non-null   float64
 2   drtg_S2D_A  3644 non-null   float64
 3   eFG_S2D_A   3644 non-null   float64
 4   tovr_S2D_A  3644 non-null   float64
 5   orb%_S2D_A  3644 non-null   float64
 6   ftr_S2D_A   3644 non-null   float64
 7   pace_S2D_A  3644 non-null   float64
dtypes: float64(8)
memory usage: 256.2 KB


In [298]:
final_y_train.info()

<class 'pandas.core.series.Series'>
Index: 3644 entries, 8 to 3731
Series name: win_int
Non-Null Count  Dtype
--------------  -----
3644 non-null   int64
dtypes: int64(1)
memory usage: 56.9 KB


In [300]:
stat_delta_val_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1222 entries, 3735 to 4979
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   poss_S2D_A  1222 non-null   float64
 1   ortg_S2D_A  1222 non-null   float64
 2   drtg_S2D_A  1222 non-null   float64
 3   eFG_S2D_A   1222 non-null   float64
 4   tovr_S2D_A  1222 non-null   float64
 5   orb%_S2D_A  1222 non-null   float64
 6   ftr_S2D_A   1222 non-null   float64
 7   pace_S2D_A  1222 non-null   float64
dtypes: float64(8)
memory usage: 85.9 KB


In [301]:
stat_delta_training_set.to_csv("stat_delta_x_train.csv", index=False)
stat_delta_val_set.to_csv("stat_delta_x_val.csv", index=False)